# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 54 new papers today
          11 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.11587


extracting tarball to tmp_2302.11587...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.11591


extracting tarball to tmp_2302.11591...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.11911


extracting tarball to tmp_2302.11911...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.11920


extracting tarball to tmp_2302.11920...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11587-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11587) | **AGN STORM 2: II. Ultraviolet Observations of Mrk817 with the Cosmic  Origins Spectrograph on the Hubble Space Telescope**  |
|| Y. Homayouni, et al. -- incl., <mark>Travis Fischer</mark>, <mark>Yan-Rong Li</mark> |
|*Appeared on*| *2023-02-24*|
|*Comments*| *Submitted to ApJ. 25 pages, 8 figures, and 6 tables*|
|**Abstract**| We present reverberation mapping measurements for the prominent ultraviolet broad emission lines of the active galactic nucleus Mrk817 using 165 spectra obtained with the Cosmic Origins Spectrograph on the Hubble Space Telescope. Our ultraviolet observations are accompanied by X-ray, optical, and near-infrared observations as part of the AGN Space Telescope and Optical Reverberation Mapping Program 2 (AGN STORM 2). Using the cross-correlation lag analysis method, we find significant correlated variations in the continuum and emission-line light curves. We measure rest-frame delayed responses between the far-ultraviolet continuum at 1180 A and Ly$\alpha$ $\lambda1215$ A ($10.4_{-1.4}^{+1.6}$ days), N V $\lambda1240$ A ($15.5_{-4.8}^{+1.0}$days), SiIV + OIV] $\lambda1397$ A ($8.2_{-1.4}^{+1.4}$ days), CIV $\lambda1549$ A ($11.8_{-2.8}^{+3.0}$ days), and HeII $\lambda1640$ A ($9.0_{-1.9}^{+4.5}$ days) using segments of the emission-line profile that are unaffected by absorption and blending, which results in sampling different velocity ranges for each line. However, we find that the emission-line responses to continuum variations are more complex than a simple smoothed, shifted, and scaled version of the continuum light curve. We also measure velocity-resolved lags for the Ly$\alpha$, and CIV emission lines. The lag profile in the blue wing of Ly$\alpha$ is consistent with virial motion, with longer lags dominating at lower velocities, and shorter lags at higher velocities. The CIV lag profile shows the signature of a thick rotating disk, with the shortest lags in the wings, local peaks at $\pm$ 1500 $\rm km\,s^{-1}$, and a local minimum at line center. The other emission lines are dominated by broad absorption lines and blending with adjacent emission lines. These require detailed models, and will be presented in future work. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11591-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11591) | **Resonant infrared irradiation of CO and CH3OH interstellar ices**  |
|| J. C. Santos, et al. -- incl., <mark>J. Zhang</mark> |
|*Appeared on*| *2023-02-24*|
|*Comments*| *Accepted for publication in A&A. 19 pages, 14 figures, 2 tables*|
|**Abstract**| Solid-phase photo-processes involving icy dust grains greatly affect the chemical evolution of the interstellar medium by leading to the formation of complex organic molecules and by inducing photodesorption. So far, the focus of laboratory studies has been mainly on the impact of energetic ultraviolet (UV) photons on ices, but direct vibrational excitation by infrared (IR) photons is expected to influence the morphology and content of interstellar ices as well. However, little is still known about the mechanisms through which this excess vibrational energy is dissipated, and its implications on the structure and ice photochemistry. In this work, we present a systematic investigation of the behavior of interstellar relevant CO and CH3OH ice analogues upon resonant excitation of vibrational modes using tunable infrared radiation, leading to both the quantification of infrared-induced photodesorption and insights in the impact of vibrational energy dissipation on ice morphology. We utilize an ultrahigh vacuum setup at cryogenic temperatures to grow pure CO and CH3OH ices, as well as mixtures of the two. We expose the ices to intense, near-monochromatic mid-infrared free-electron-laser radiation to selectively excite the species. The dissipation of vibrational energy is observed to be highly dependent on the excited mode and the chemical environment of the ice. All amorphous ices undergo some degree of restructuring towards a more organized configuration upon on-resonance irradiation. Moreover, IR-induced photodesorption is observed to occur for both pure CO and CH3OH ices, with interstellar photodesorption efficiencies of the order of 10 molecules cm-2 s-1 (i.e., comparable to or higher than UV-induced counterparts). Indirect photodesorption of CO upon vibrational excitation of CH3OH in ice mixtures is also observed to occur, particularly in environments rich in methanol. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11621-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11621) | **Isolating the linear signal when making redshift space distortion  measurements**  |
|| Michael J. Chapman, <mark>Zhongxu Zhai</mark>, Will J. Percival |
|*Appeared on*| *2023-02-24*|
|*Comments*| *14 pages, 9 figures, submitted to MNRAS*|
|**Abstract**| Constraints on the linear growth rate, $f\sigma_8$, using small scale redshift space distortion measurements have a significant statistical advantage over those made on large scales. However, these measurements need to carefully disentangle the linear and non-linear information when interpreting redshift space distortions in terms of $f\sigma_8$. It is particularly important to do this given that some previous measurements found a significant deviation from the expectation based on the $\Lambda$CDM model constrained by Planck CMB data. We construct a new emulator-based model for small scale galaxy clustering with scaling parameters for both the linear and non-linear velocities of galaxies, allowing us to isolate the linear growth rate. We train the emulator using simulations from the AbacusCosmos suite, and apply it to data from the extended Baryon Oscillation Spectroscopic Survey (eBOSS) luminous red galaxy sample. We obtain a value of $f\sigma_8(z=0.737)=0.368\pm0.041$, in 2.3-$\sigma$ tension with the Planck 2018 $\Lambda$CDM expectation, and find less dependence on the minimum measurement scale than previous analyses. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11760-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11760) | **Absolute Properties of the Oscillating Eclipsing Algol X Trianguli**  |
|| <mark>Jae Woo Lee</mark>, et al. |
|*Appeared on*| *2023-02-24*|
|*Comments*| *27 pages, including 8 figures and 6 tables, accepted for publication in AJ*|
|**Abstract**| We report results from the TESS photometric data and new high-resolution spectra of the Algol system X Tri showing short-period pulsations. From the echelle spectra, the radial velocities of the eclipsing pair were measured, and the rotational rate and effective temperature of the primary star were obtained to be $v_1$$\sin$$i=84\pm6$ km s$^{-1}$ and $T_{\rm eff,1}=7900 \pm 110$ K, respectively. The synthetic modeling of these observations implies that X Tri is in synchronous rotation and is physically linked to a visual companion TIC 28391715 at a separation of about 6.5 arcsec. The absolute parameters of our target star were accurately and directly determined to be $M_1 = 2.137 \pm 0.018$ $M_\odot$, $M_2 = 1.101 \pm 0.010$ $M_\odot$, $R_1 = 1.664 \pm 0.010$ $R_\odot$, $R_2 = 1.972 \pm 0.010$ $R_\odot$, $L_1 = 9.67 \pm 0.55$ $L_\odot$, and $L_2 = 2.16 \pm 0.09$ $L_\odot$. The phase-binned mean light curve was used to remove the binary effect from the observed TESS data. Multifrequency analysis of the residuals revealed 16 significant frequencies, of which the high-frequency signals between 37 day$^{-1}$ and 48 day$^{-1}$ can be considered probable pulsation modes. Their oscillation periods of 0.021$-$0.027 days and pulsation constants of 0.014$-$0.018 days are typical values of $\delta$ Sct variables. The overall results demonstrate that X Tri is an oEA star system, consisting of a $\delta$ Sct primary and its lobe-filling companion in the semi-detached configuration. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11792-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11792) | **Broadband X-ray Spectroscopy of the Pulsar Wind Nebula in HESS J1640-465**  |
|| Moaz Abdelmaguid, <mark>Joseph D Gelfand</mark>, Eric Gotthelf, Samayra Straal |
|*Appeared on*| *2023-02-24*|
|*Comments*| *14 pages, 7 figures, 10 tables. Accepted for publication in ApJ*|
|**Abstract**| We present updated measurements of the X-ray properties of the pulsar wind nebula associated with the TeV $\gamma$-ray source HESS J1640-465 derived from Chandra & NuSTAR data. We report a high $N_{H}$ value along line of sight, consistent with previous work, which led us to incorporate effects of dust scattering in our spectral analysis. Due to uncertainties in the dust scattering, we report a range of values for the PWN properties (photon index and un-absorbed flux). In addition, we fit the broadband spectrum of this source and found evidence for spectral softening and decreasing unabsorbed flux as we go to higher photon energies. We then used a one zone time dependent evolutionary model to reproduce the dynamical and multi-wavelength spectral properties of our source. Our model suggests a short spin-down time scale, a relatively higher than average magnetized pulsar wind, a strong PWN magnetic field and maximum electron energy up to PeV, suggesting HESS J1640-465 could be a PeVatron candidate. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11895-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11895) | **The MeerKAT Fornax Survey -- I. Survey description and first evidence of  ram pressure in the Fornax galaxy cluster**  |
|| P. Serra, et al. -- incl., <mark>B. S. Frank</mark>, <mark>M. W. L. Smith</mark> |
|*Appeared on*| *2023-02-24*|
|*Comments*| *Astronomy & Astrophysics, accepted. Data available at the MeerKAT Fornax Survey website this https URL*|
|**Abstract**| The MeerKAT Fornax Survey maps the distribution and kinematics of atomic neutral hydrogen gas (HI) in the nearby Fornax galaxy cluster using the MeerKAT telescope. The 12 deg^2 survey footprint covers the central region of the cluster out to ~ Rvir and stretches out to ~ 2 Rvir towards south west to include the NGC 1316 galaxy group. The HI column density sensitivity (3 sigma over 25 km/s) ranges from 5e+19/cm^2 at a resolution of ~ 10" (~ 1 kpc at the 20 Mpc distance of Fornax) down to ~ 1e+18/cm^2 at ~ 1' (~ 6 kpc), and slightly below this level at the lowest resolution of ~ 100" (~ 10 kpc). The HI mass sensitivity (3 sigma over 50 km/s) is 6e+5 Msun. The HI velocity resolution is 1.4 km/s. In this paper we describe the survey design and HI data processing, and we present a sample of six galaxies with long, one-sided, star-less HI tails (of which only one was previously known) radially oriented within the cluster and with measurable internal velocity gradients. We argue that the joint properties of the HI tails represent the first unambiguous evidence of ram pressure shaping the distribution of HI in the Fornax cluster. The disturbed optical morphology of all host galaxies supports the idea that the tails consist of HI initially pulled out of the galaxies' stellar body by tidal forces. Ram pressure was then able to further displace the weakly bound HI and give the tails their present direction, length and velocity gradient. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11911-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11911) | **The Southern-sky MWA Rapid Two-metre (SMART) pulsar survey -- I. Survey  design and processing pipeline**  |
|| N. D. R. Bhat, et al. -- incl., <mark>K. R. Smith</mark> |
|*Appeared on*| *2023-02-24*|
|*Comments*| *22 pages, 12 figures, 2 tables, Accepted for publication in PASA*|
|**Abstract**| We present an overview of the Southern-sky MWA Rapid Two-metre (SMART) pulsar survey that exploits the MWA's large field of view and voltage capture system to survey the sky south of 30 degree in declination for pulsars and fast transients in the 140-170 MHz band. The survey is enabled by the advent of the Phase II MWA's compact configuration, which offers an enormous efficiency in beam-forming and processing costs, thereby making an all-sky survey of this magnitude tractable with the MWA. Even with the long dwell times of the survey (4800 s), data collection can be completed in < 100 hours of telescope time, while still retaining the ability to reach a limiting sensitivity of ~2-3 mJy. Each observation is processed to generate ~5000-8000 tied-array beams that tessellate the full ~610 square degree field of view, which are then processed to search for pulsars. The voltage-capture recording allows a multitude of post hoc processing options including the reprocessing of data for higher time resolution. Due to the substantial computational cost in pulsar searches at low frequencies, processing is undertaken in multiple passes: in the first pass, a shallow survey is performed, where 10 minutes of each observation is processed, reaching about one-third of the full search sensitivity. Here we present the system overview and initial results. Further details including first pulsar discoveries and a census of low-frequency detections are presented in a companion paper. Future plans include deeper searches to reach the full sensitivity and acceleration searches to target binary and millisecond pulsars. Simulation analysis forecasts ~300 new pulsars upon the completion of full processing. The SMART survey will also generate a complete digital record of the low-frequency sky, which will serve as a valuable reference for future pulsar searches planned with the low-frequency Square Kilometre Array. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.11920-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.11920) | **The Southern-sky MWA Rapid Two-metre (SMART) pulsar survey -- II. Survey  status, pulsar census, and first pulsar discoveries**  |
|| N. D. R. Bhat, et al. -- incl., <mark>K. R. Smith</mark> |
|*Appeared on*| *2023-02-24*|
|*Comments*| *22 pages, 9 figures, 7 tables, Accepted for publication in PASA*|
|**Abstract**| In Paper I, we presented an overview of the Southern-sky MWA Rapid Two-metre (SMART) survey, including the survey design and search pipeline. While the combination of MWA's large field-of-view and the voltage capture system brings a survey speed of ~450 square degrees per hour, the survey progression relies on the availability of compact configuration of the Phase II array. Over the past few years, by taking advantage of multiple windows of opportunity when the compact configuration was available, we have advanced the survey to 75% completion. To date, about 10% of the data collected thus far have been processed for a first-pass search, where 10 minutes of observation is processed for dispersion measures out to 250 ${\rm pc\,cm^{-3}}$, to realise a shallow survey for long-period pulsars. The ongoing analysis has led to two new pulsar discoveries, as well as an independent discovery and a rediscovery of a previously incorrectly characterised pulsar, all from ~3% of the data for which candidate scrutiny is completed. Here we describe the strategies for further detailed follow-up including improved sky localisation and convergence to timing solution, and illustrate them using example pulsar discoveries. The processing has also led to re-detection of 120 pulsars in the SMART observing band, bringing the total number of pulsars detected to date with the MWA to 180, and these are used to assess the search sensitivity of current processing pipelines. The planned second-pass (deep survey) processing is expected to yield a three-fold increase in sensitivity for long-period pulsars, and a substantial improvement to millisecond pulsars by adopting optimal de-dispersion plans. The SMART survey will complement the highly successful Parkes High Time Resolution Universe survey at 1.2-1.5 GHz, and inform future large survey efforts such as those planned with the low-frequency Square Kilometre Array. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

213  publications in the last 7 days.
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


# Debugging papers